In [2]:
import torch
import cv2
import numpy as np
import yaml
from yaml.loader import SafeLoader
import time
from collections import defaultdict
from ultralytics import YOLO

In [3]:
BEST_PT_PATH = "D:\VS Code Folders\Squash-Ball-Detection\Ball Tracking\Results 4500 images dataset\\results\weights\\best.pt"
DATA_YAML_PATH = 'D:\VS Code Folders\Squash-Ball-Detection\data.yaml'
video_path = 'D:\VS Code Folders\Squash-Ball-Detection\Greatest Squash Trickshots.mp4'


In [71]:
# Load the YOLOv8 model
model = YOLO(BEST_PT_PATH)

# Open the video file
videoCapture = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while videoCapture.isOpened():
    # Read a frame from the video
    success, frame = videoCapture.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, conf=0.25, iou=0.5, show=False)

        if results and results[0].boxes:
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            results[0].names[0] = 'ball'
            if results[0].boxes.id is not None:
                track_ids = results[0].boxes.id.cpu().tolist()
                confidences = results[0].boxes.conf.cpu().tolist()
            else:
                track_ids = []

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            # annotated_frame = frame.copy()
            # Plot the tracks
            for box, track_id, confidence in zip(boxes, track_ids, confidences):
                # if confidence > 0.25:
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 90:  # Retain tracks for 90 frames
                    track.pop(0)

                # annotated_frame = results[0].plot()
                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=2)
                # cv2.rectangle(annotated_frame, (int(x-w/2), int(y-h/2)), (int(x+w/2), int(y+h/2)), (0, 255, 0), 2)
                # # Display track ID and confidence score
                # cv2.putText(annotated_frame, f'ID: {track_id} Conf: {confidence:.2f}', (int(x), int(y) - 10),
                #             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
videoCapture.release()
cv2.destroyAllWindows()



0: 384x640 1 Squash_Ball detection - v1 2023-12-03 1-09pm, 572.0ms
Speed: 13.0ms preprocess, 572.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 balls, 607.4ms
Speed: 36.3ms preprocess, 607.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 385.9ms
Speed: 5.0ms preprocess, 385.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 386.5ms
Speed: 4.1ms preprocess, 386.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 384.9ms
Speed: 4.0ms preprocess, 384.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 425.3ms
Speed: 5.0ms preprocess, 425.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 411.1ms
Speed: 5.1ms preprocess, 411.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 380.0ms
Speed: 4.0ms preproc

In [58]:
result = results[0]
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'ball'}
 obb: None
 orig_img: array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
     

In [27]:
type(results)

list